In [76]:
# Plot H0 Q-Q plots

import pickle
import pylab 
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np

In [77]:
def PlotH0(Model, Index, n, Lapscale, sig, Xdist, testtype, fixedU, randomBeta, Uind):
    fname = Model + "_Setting_" + str(Index) + "_n_" + str(n) + "_Lapscale_" + str(Lapscale) + "_sig_" + str(sig) + "_Xdist_" + str(Xdist) + "_testtype_" + testtype + "_fixedU_" + str(fixedU) + "_randomBeta_" + str(randomBeta) + "_Uind_" + str(Uind)

    infile = open( Model + "_results/resultsH0/data/" + fname + '_dic.p', 'rb')
    new_dict = pickle.load(infile)
    infile.close()
    pV_array = (new_dict["pV"])

    if sig == 0:
        rhoName = "Independent"
    else:
        rhoName = 'AR(1) ' + r'$\rho=' +str(sig) + '$'
    SettingName = 'Setting ' + str(Index) + ', n=' + str(n) +', ' + rhoName + ', ' + r'$lap(' +str(Lapscale) + ')$' + ', ' + r'$U_' +str((Uind+1)) + '$'


    fig = plt.figure()
    fig.suptitle(SettingName, fontsize=20)
    for t in range(1, 6):
        ax = fig.add_subplot(int('15' + str(t)))
        res = stats.probplot(pV_array[:,(t-1)], dist="uniform", plot=pylab)
        ax.set_title('t=' + str(t), fontsize=18)
        ax.set_ylabel('')
        ax.set_xlabel('')
    fig.set_size_inches(11.2, 3.2)   
    fig.text(0.5,0.04, "Theoretical Quantiles", ha="center", va="center", fontsize=20)
    fig.text(0.025,0.5, "Sample Quantiles", ha="center", va="center", rotation=90, fontsize=17)
    fig.tight_layout(rect=[0.05, 0.07, 1, 0.95])
    plt.savefig(Model + "_results/Result_Figure/ResultsH0" + fname +'.pdf')

In [78]:
# Model = "logistic"
# Index = 1
# n = 300
# Lapscale = 0
Xdist = "uniform"
# sig = 0
testtype = "Wald"

randomBeta = False

In [79]:
Model = "logistic"
Index = 1
n = 2000
Lapscale = 0.25
sig = 0.1
fixedU = ""

# Number of fixed U matrices
nU = 6

# significance level
alpha = 0.05

hypothesis = "H0"

In [80]:
LapscaleList = [0, 0.1, 0.5]

In [83]:
def GenCSV(LapscaleList, alpha, hypothesis, Model):
    for LapscaleInd in range(len(LapscaleList)):
        Lapscale = LapscaleList[LapscaleInd]
        for Uind in range(nU):
            if hypothesis == "H0":
                randomBeta = False
            elif hypothesis == "H1":
                randomBeta = True
            fname = Model + "_Setting_" + str(Index) + "_n_" + str(n) + "_Lapscale_" + str(Lapscale) + "_sig_" + str(sig) + "_Xdist_" + str(Xdist) + "_testtype_" + testtype + "_fixedU_" + str(fixedU) + "_randomBeta_" + str(randomBeta) + "_Uind_" + str(Uind)

            infile = open( Model + "_results/results" + hypothesis + "/data/" + fname + '_dic.p', 'rb')
            new_dict = pickle.load(infile)
            infile.close()
            pV_array = (new_dict["pV"])

            # append the rejection results
            if Uind == 0:
                #pV_array_reps = pV_array.reshape(-1, pV_array.shape[0], pV_array.shape[1])
                pV_array_rej = (pV_array < alpha)*1
            elif Uind > 0:
                #pV_array_reps = np.concatenate([pV_array_reps,pV_array.reshape(-1, pV_array.shape[0], pV_array.shape[1])], axis=0)
                pV_array_rej = pV_array_rej + (pV_array < alpha)*1

        # calculate the number of matches
        match_results = (pV_array_rej == 0) + (pV_array_rej == 6)

        if LapscaleInd == 0:
            match_counts = np.sum(match_results * 1, axis = 0).reshape(1,-1)
        elif LapscaleInd> 0:
            match_counts = np.concatenate((match_counts, np.sum(match_results * 1, axis = 0).reshape(1,-1)), axis = 0)
        
        match_counts.astype(int)
        np.savetxt(Model + "_" + hypothesis + ".csv", match_counts, delimiter=",")

In [84]:
for Model in ["logistic", "normal", "poisson", "logcosh"]:
    for hypothesis in ["H0", "H1"]:
        GenCSV(LapscaleList, alpha, hypothesis, Model)